<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Sprint Challenge
## *Data Science Unit 4 Sprint 1*

After a week of Natural Language Processing, you've learned some cool new stuff: how to process text, how turn text into vectors, and how to model topics from documents. Apply your newly acquired skills to one of the most famous NLP datasets out there: [Yelp](https://www.yelp.com/dataset/challenge). As part of the job selection process, some of my friends have been asked to create analysis of this dataset, so I want to empower you to have a head start.  

The real dataset is massive (almost 8 gigs uncompressed). I've sampled the data for you to something more managable for the Sprint Challenge. You can analyze the full dataset as a stretch goal or after the sprint challenge. As you work on the challenge, I suggest adding notes about your findings and things you want to analyze in the future.

## Challenge Objectives
*Successfully complete these all these objectives to earn a 2. There are more details on each objective further down in the notebook.*
* <a href="#p1">Part 1</a>: Write a function to tokenize the yelp reviews
* <a href="#p2">Part 2</a>: Create a vector representation of those tokens
* <a href="#p3">Part 3</a>: Use your tokens in a classification model on yelp rating
* <a href="#p4">Part 4</a>: Estimate & Interpret a topic model of the Yelp reviews

In [98]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd

yelp = pd.read_json('./data/review_sample.json', lines=True)

In [3]:
yelp.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,nDuEqIyRc8YKS1q1fX0CZg,1,2015-03-31 16:50:30,0,eZs2tpEJtXPwawvHnHZIgQ,1,"BEWARE!!! FAKE, FAKE, FAKE....We also own a sm...",10,n1LM36qNg4rqGXIcvVXv8w
1,eMYeEapscbKNqUDCx705hg,0,2015-12-16 05:31:03,0,DoQDWJsNbU0KL1O29l_Xug,4,Came here for lunch Togo. Service was quick. S...,0,5CgjjDAic2-FAvCtiHpytA
2,6Q7-wkCPc1KF75jZLOTcMw,1,2010-06-20 19:14:48,1,DDOdGU7zh56yQHmUnL1idQ,3,I've been to Vegas dozens of times and had nev...,2,BdV-cf3LScmb8kZ7iiBcMA
3,k3zrItO4l9hwfLRwHBDc9w,3,2010-07-13 00:33:45,4,LfTMUWnfGFMOfOIyJcwLVA,1,We went here on a night where they closed off ...,5,cZZnBqh4gAEy4CdNvJailQ
4,6hpfRwGlOzbNv7k5eP9rsQ,1,2018-06-30 02:30:01,0,zJSUdI7bJ8PNJAg4lnl_Gg,4,"3.5 to 4 stars\n\nNot bad for the price, $12.9...",5,n9QO4ClYAS7h9fpQwa5bhA


## Part 1: Tokenize Function
<a id="#p1"></a>

Complete the function `tokenize`. Your function should
- accept one document at a time
- return a list of tokens

You are free to use any method you have learned this week.

In [4]:
import spacy
from spacy.tokenizer import Tokenizer

In [10]:
# Load Spacy model & tokenizer
nlp = spacy.load("en_core_web_lg")
tokenizer = Tokenizer(nlp.vocab)

In [112]:
# Define Stop Words
STOP_WORDS = nlp.Defaults.stop_words.union([' ', '  ', 'the', 'like', 'i', 'for', 'I', "i've", 'we', '\n', '\n\n', 'my', 'this'])

In [113]:
# Initialize tokens list
tokens = []

In [114]:
def tokenize(series):
    for doc in tokenizer.pipe(series, batch_size = 500):
        # Initialize doc tokens
        doc_tokens = []

        # Filter for stop words
        for token in doc:
            if token.text not in STOP_WORDS:
                doc_tokens.append(token.text.lower())

        # Append doc_tokens to tokens
        tokens.append(doc_tokens)
    return tokens

In [115]:
yelp['tokens'] = tokenize(yelp['text'])

In [119]:
# Clean up the stragglers
for token_list in yelp['tokens']:
    for token in token_list:
        if token in STOP_WORDS:
            token_list.remove(token)

In [120]:
yelp['tokens'].tail()

9995    [family, hungry, subway, open, 24, hours, guy,...
9996    [wife, came, couple, friends., sever, excited,...
9997    [food, ok, brag, about., food, hot,, items, ta...
9998    [today's, visit, great!!, love, enjoy, town, s...
9999    [absolute, worst, place, stayed, 43, year, lif...
Name: tokens, dtype: object

## Part 2: Vector Representation
<a id="#p2"></a>
1. Create a vector representation of the reviews
2. Write a fake review and query for the 10 most similiar reviews, print the text of the reviews. Do you notice any patterns?
    - Given the size of the dataset, it will probably be best to use a `NearestNeighbors` model for this. 

In [169]:
# Make some clean text for the TFIDF vectorizer
# The built-in stop words function might not work very well
clean_text = []

for token_list in yelp['tokens']:
    clean_review = ' '.join(token_list)
    clean_text.append(clean_review)

In [172]:
# Send clean text to df
yelp['clean_text'] = clean_text

In [121]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [122]:
# Instantiate Vectorizer
tfidf = TfidfVectorizer(stop_words = 'english')

In [173]:
# Create vocab and get word counts
sparse = tfidf.fit_transform(yelp['clean_text'])

In [174]:
# send the matrix to a DataFrame
tfidf_dtm = pd.DataFrame(sparse.todense(), columns = tfidf.get_feature_names())

In [175]:
tfidf_dtm.shape

(10000, 27287)

In [176]:
# Import NN Model
from sklearn.neighbors import NearestNeighbors

In [195]:
# Instantiate nearest neighbors model
nn = NearestNeighbors(n_neighbors=5, algorithm='ball_tree')
nn.fit(tfidf_dtm)

NearestNeighbors(algorithm='ball_tree', leaf_size=30, metric='minkowski',
         metric_params=None, n_jobs=None, n_neighbors=5, p=2, radius=1.0)

In [196]:
# Create a fake review
fake_review = 'this product did not work at all. I cannot believe I wasted so much money on it.'
fake = pd.Series(fake_review)

In [198]:
# Transform
fake_sparse = tfidf.transform(fake)

# Densify
fake_tfidf = pd.DataFrame(fake_sparse.todense())

In [229]:
nn.kneighbors([fake_tfidf][0], n_neighbors=10)

(array([[1.22301302, 1.2729863 , 1.28512821, 1.28770449, 1.29229009,
         1.29558748, 1.29819142, 1.30206758, 1.30268916, 1.30916134]]),
 array([[8956, 8540, 5666, 4203, 2434, 3209, 8902, 5630, 1132, 6427]]))

In [230]:
print(yelp['text'][8956])
print('')
print(yelp['text'][8540])
print('')
print(yelp['text'][5666])
print('')
print(yelp['text'][4203])
print('')
print(yelp['text'][2434])
print('')
print(yelp['text'][3209])
print('')
print(yelp['text'][8902])
print('')
print(yelp['text'][5630])
print('')
print(yelp['text'][1132])
print('')
print(yelp['text'][6427])
print('')

Save your money and time! Horrible customer services and product is garbage. Tip run and never look be back!

I needed a money order. Went into 7-Eleven they gave quick and service. I was in and out with my money order and a drink 2 minutes.

Yeah, it's July 4 and you are out of lighter fluid and deer spray.  Really, you need to order more stuff.  Isn't July 4 big for barbeque.  Can't you check what you sold last year and order up.  Everyone here has deer issues. what's up with that??? Lots of other shelves empty.  Looks like you cut payroll hours to save money and lost money!   Wasted trip here.

Bought a dozen roses and they were beautiful.  Service was quick and the product was amazing.  Price wasn't too bad either.

The worst dinner ever
Maybe worse than the last time we were there.
Time to re evaluate the bf who keeps selecting this place.
Never again for me
Epic fail 
Poor everything
I will never set foot in the door again
Embarrassing to take guests there
But he knew of the fail

The first few mention 'money' which is in my fake review. Some of the results are pretty positive, compared to my negative
fake review which is surprising. I see some mention 'product' which I used since I wasn't reviewing a real product.
I think some of the positive reviews snuck in there based off of key words like 'money' and 'product' which aren't
mentioned often. Perhaps noun-chunks would capture sentiment better. Then I could find things like 'bad service' or 
'delicious food'.

## Part 3: Classification
<a id="#p3"></a>
Your goal in this section will be to predict `stars` from the review dataset. 

1. Create a piepline object with a sklearn `CountVectorizer` or `TfidfVector` and any sklearn classifier. Use that pipeline to estimate a model to predict `stars`. Use the Pipeline to predict a star rating for your fake review from Part 2. 
2. Tune the entire pipeline with a GridSearch

In [134]:
# Import Statements
from sklearn.pipeline import Pipeline

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier

In [135]:
# Create pipeline
vect = CountVectorizer(stop_words = 'english')
rfc = RandomForestClassifier()

pipe = Pipeline([('vect', vect), ('rfc', rfc)])

In [136]:
# Fit Pipeline
pipe.fit(yelp['text'], yelp['stars'])

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words='english',
        ...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

In [137]:
# I wrote a bad review
pipe.predict([fake_review])[0]

1

In [138]:
# Here's a good one
good_review = '''this is the most amazing product I have ever used. I will never go anywhere without this widget
 my life has been permanently changed for the better'''

In [139]:
# Good one
pipe.predict([good_review])[0]

5

## Tune the Pipeline

In [201]:
# Import
from sklearn.model_selection import GridSearchCV

In [206]:
# Set params for grid search
parameters = {
    'vect__max_df': (0.5, 0.75, 1.0),
    'vect__min_df': (.02, .05),
    'vect__max_features': (100, 500,1000)}

In [207]:
# Set up and call Grid Search
grid_search = GridSearchCV(pipe,parameters, cv=3, n_jobs=-1, verbose=1)
grid_search.fit(yelp['text'], yelp['stars'])

Fitting 3 folds for each of 18 candidates, totalling 54 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   40.8s
[Parallel(n_jobs=-1)]: Done  54 out of  54 | elapsed:   50.2s finished


GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words='english',
        ...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'vect__max_df': (0.5, 0.75, 1.0), 'vect__min_df': (0.02, 0.05), 'vect__max_features': (100, 500, 1000)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [208]:
# Report best params
grid_search.best_params_

{'vect__max_df': 0.5, 'vect__max_features': 1000, 'vect__min_df': 0.02}

In [211]:
# Set params for grid search
parameters = {
    'rfc__n_estimators': (10, 1000, 100),
    'rfc__max_depth': (2, 100, 10)}

In [213]:
# Set up and call Grid Search
grid_search = GridSearchCV(pipe,parameters, cv=3, n_jobs=-1, verbose=1)
grid_search.fit(yelp['text'], yelp['stars'])

Fitting 3 folds for each of 9 candidates, totalling 27 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  27 out of  27 | elapsed:  2.6min finished


GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words='english',
        ...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'rfc__n_estimators': (10, 1000, 100), 'rfc__max_depth': (2, 100, 10)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [214]:
# Report best params
grid_search.best_params_

{'rfc__max_depth': 100, 'rfc__n_estimators': 1000}

In [215]:
# Retune pipeline
vect = CountVectorizer(stop_words='english', max_df=0.5, min_df=0.02, max_features=1000)
rfc = RandomForestClassifier(max_depth=100, n_estimators=1000)

pipe = Pipeline([('vect', vect), ('rfc', rfc)])

In [216]:
# Refit Pipeline
pipe.fit(yelp['text'], yelp['stars'])

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.5, max_features=1000, min_df=0.02,
        ngram_range=(1, 1), preprocessor=None, stop_words='english',
     ...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

## Part 4: Topic Modeling

Let's find out what those yelp reviews are saying! :D

1. Estimate a LDA topic model of the review text
    - Keep the `iterations` parameter at or below 5 to reduce run time
    - The `workers` parameter should match the number of physical cores on your machine.
2. Create 1-2 visualizations of the results
    - You can use the most important 3 words of a topic in relevant visualizations. Refer to yesterday's notebook to extract. 
3. In markdown, write 1-2 paragraphs of analysis on the results of your topic model

__*Note*__: You can pass the DataFrame column of text reviews to gensim. You do not have to use a generator.

In [140]:
from gensim.models import LdaMulticore
from gensim.corpora import Dictionary

Learn the vocubalary of the yelp data:

In [141]:
id2word = Dictionary(tokens)

In [220]:
# Filter extreme values
id2word.filter_extremes(no_below=100, no_above=0.85)

Create a bag of words representation of the entire corpus

In [221]:
corpus = [id2word.doc2bow(text) for text in tokens]

Your LDA model should be ready for estimation: 

In [222]:
lda = LdaMulticore(corpus=corpus,
                   id2word=id2word,
                   iterations=5,
                   workers=4,
                   num_topics =10 # You can change this parameter
                  )

Create 1-2 visualizations of the results

In [223]:
import pyLDAvis.gensim

In [224]:
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(lda, corpus, id2word)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
8      0.005803  0.002738       1        1  10.421406
2      0.004016  0.003089       2        1  10.265140
6     -0.001763 -0.007119       3        1  10.063681
5      0.005235 -0.001296       4        1  10.054570
4     -0.000560  0.004367       5        1   9.942924
3     -0.000095 -0.002451       6        1   9.932015
9     -0.001186  0.003304       7        1   9.893760
0      0.001261 -0.003953       8        1   9.828876
7     -0.009484  0.002986       9        1   9.810070
1     -0.003227 -0.001666      10        1   9.787560, topic_info=    Category         Freq        Term        Total  loglift  logprob
63   Default  3721.000000       place  3721.000000  30.0000  30.0000
101  Default  3398.000000       great  3398.000000  29.0000  29.0000
88   Default  3065.000000        good  3065.000000  28.0000  28.0000
360  Default  2656.000000        it's  2656.000000  27.0000  27.0000
179  Default  3624.000000        food  3624.000000  26.0000  26.0000
20   Default  1476.000000        came  1476.000000  25.0000  25.0000
35   Default  2021.000000         got  2021.000000  24.0000  24.0000
186  Default  1171.000000       order  1171.000000  23.0000  23.0000
83   Default  1780.000000           -  1780.000000  22.0000  22.0000
55   Default  1467.000000        went  1467.000000  21.0000  21.0000
247  Default   900.000000    friendly   900.000000  20.0000  20.0000
91   Default  1052.000000   recommend  1052.000000  19.0000  19.0000
144  Default  1254.000000         try  1254.000000  18.0000  18.0000
97   Default  1815.000000       don't  1815.000000  17.0000  17.0000
17   Default  2477.000000        time  2477.000000  16.0000  16.0000
26   Default  2575.000000     service  2575.000000  15.0000  15.0000
303  Default  1509.000000      didn't  1509.000000  14.0000  14.0000
173  Default  1453.000000  definitely  1453.000000  13.0000  13.0000
165  Default   700.000000     amazing   700.000000  12.0000  12.0000
199  Default   925.000000        took   925.000000  11.0000  11.0000
78   Default  1034.000000      pretty  1034.000000  10.0000  10.0000
201  Default   812.000000           2   812.000000   9.0000   9.0000
58   Default  1637.000000        best  1637.000000   8.0000   8.0000
126  Default   856.000000  experience   856.000000   7.0000   7.0000
477  Default  1458.000000     ordered  1458.000000   6.0000   6.0000
4    Default   618.000000      called   618.000000   5.0000   5.0000
22   Default   840.000000       good.   840.000000   4.0000   4.0000
189  Default   570.000000       pizza   570.000000   3.0000   3.0000
171  Default  1433.000000        come  1433.000000   2.0000   2.0000
15   Default  1099.000000        said  1099.000000   1.0000   1.0000
..       ...          ...         ...          ...      ...      ...
303  Topic10   190.495987      didn't  1509.993896   0.2538  -4.8921
657  Topic10    46.289467        too.   338.381165   0.3348  -6.3069
173  Topic10   181.416809  definitely  1453.832764   0.2429  -4.9410
15   Topic10   138.639069        said  1099.243042   0.2536  -5.2099
242  Topic10   120.798721        told   968.239502   0.2427  -5.3476
69   Topic10    82.558861   different   641.147827   0.2743  -5.7283
267  Topic10    52.045265        line   385.440399   0.3218  -6.1897
360  Topic10   293.373840        it's  2656.066650   0.1209  -4.4603
470  Topic10    69.443527       time.   534.613220   0.2830  -5.9013
101  Topic10   351.938782       great  3398.891602   0.0563  -4.2783
89   Topic10   103.703041        menu   855.173279   0.2143  -5.5002
88   Topic10   307.693817        good  3065.626709   0.0252  -4.4127
91   Topic10   123.502502   recommend  1052.141846   0.1817  -5.3255
17   Topic10   245.897034        time  2477.808105   0.0138  -4.6369
186  Topic10   132.282379       order  1171.494873   0.1430  -5.2568
144  Topic10   139.192398         try  1254.036377   0.1258  -5.20

In [225]:
lda = LdaMulticore(corpus=corpus,
                   id2word=id2word,
                   iterations=5,
                   workers=4,
                   num_topics =7 # You can change this parameter
                  )

In [226]:
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(lda, corpus, id2word)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.007336 -0.002004       1        1  14.451146
0      0.003839 -0.007391       2        1  14.388597
3      0.000238  0.001295       3        1  14.359501
5      0.001259  0.003277       4        1  14.282518
1     -0.001353  0.001996       5        1  14.238861
2     -0.000714 -0.000092       6        1  14.148815
6      0.004067  0.002919       7        1  14.130561, topic_info=    Category         Freq        Term        Total  loglift  logprob
101  Default  3418.000000       great  3418.000000  30.0000  30.0000
91   Default  1069.000000   recommend  1069.000000  29.0000  29.0000
88   Default  3103.000000        good  3103.000000  28.0000  28.0000
83   Default  1785.000000           -  1785.000000  27.0000  27.0000
17   Default  2506.000000        time  2506.000000  26.0000  26.0000
242  Default   969.000000        told   969.000000  25.0000  25.0000
360  Default  2666.000000        it's  2666.000000  24.0000  24.0000
58   Default  1640.000000        best  1640.000000  23.0000  23.0000
205  Default   901.000000      better   901.000000  22.0000  22.0000
104  Default  1152.000000         it.  1152.000000  21.0000  21.0000
55   Default  1454.000000        went  1454.000000  20.0000  20.0000
179  Default  3615.000000        food  3615.000000  19.0000  19.0000
27   Default  1256.000000       staff  1256.000000  18.0000  18.0000
263  Default   670.000000       found   670.000000  17.0000  17.0000
354  Default  1016.000000           &  1016.000000  16.0000  16.0000
63   Default  3749.000000       place  3749.000000  15.0000  15.0000
100  Default  1118.000000       going  1118.000000  14.0000  14.0000
39   Default   644.000000        long   644.000000  13.0000  13.0000
173  Default  1457.000000  definitely  1457.000000  12.0000  12.0000
348  Default   433.000000         ice   433.000000  11.0000  11.0000
97   Default  1812.000000       don't  1812.000000  10.0000  10.0000
26   Default  2571.000000     service  2571.000000   9.0000   9.0000
89   Default   853.000000        menu   853.000000   8.0000   8.0000
225  Default  1110.000000        know  1110.000000   7.0000   7.0000
484  Default   710.000000         lot   710.000000   6.0000   6.0000
180  Default   691.000000       fresh   691.000000   5.0000   5.0000
363  Default   487.000000       store   487.000000   4.0000   4.0000
413  Default   527.000000      that's   527.000000   3.0000   3.0000
185  Default  1427.000000      little  1427.000000   2.0000   2.0000
300  Default   419.000000       back.   419.000000   1.0000   1.0000
..       ...          ...         ...          ...      ...      ...
88    Topic7   483.204468        good  3103.188965   0.0971  -4.3286
17    Topic7   398.873718        time  2506.754883   0.1187  -4.5204
68    Topic7   140.803635         bit   793.301025   0.2280  -5.5616
435   Topic7   184.552628         new  1086.921753   0.1837  -5.2911
35    Topic7   310.048279         got  2044.403809   0.0707  -4.7723
63    Topic7   514.251892       place  3749.243652  -0.0298  -4.2663
7     Topic7    95.898941        left   522.498474   0.2615  -5.9457
330   Topic7   241.893555        love  1585.340210   0.0768  -5.0205
265   Topic7   238.238541         i'm  1559.188843   0.0782  -5.0357
62    Topic7   188.922272      people  1185.466187   0.1203  -5.2677
199   Topic7   153.428558        took   927.109985   0.1580  -5.4758
91    Topic7   172.173584   recommend  1069.282104   0.1306  -5.3605
201   Topic7   135.529099           2   809.058838   0.1701  -5.5998
360   Topic7   339.098358        it's  2666.838135  -0.1055  -4.6827
185   Topic7   207.667847      little  1427.970825   0.0288  -5.1731
179   Topic7   396.453674        food  3615.766357  -0.2537  -4.5264
203   Topic7   133.687302       asked   809.390259   0.1561  -5.6135
225   Topic7   165.835938        know  1110.833984   0.0550  -5.3980
144   Topic7   179.68

In [227]:
lda = LdaMulticore(corpus=corpus,
                   id2word=id2word,
                   iterations=5,
                   workers=4,
                   num_topics =2 # You can change this parameter
                  )

In [228]:
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(lda, corpus, id2word)

PreparedData(topic_coordinates=              x    y  topics  cluster       Freq
topic                                           
0      0.004665  0.0       1        1  50.058701
1     -0.004665  0.0       2        1  49.941299, topic_info=    Category         Freq        Term        Total  loglift  logprob
26   Default  2599.000000     service  2599.000000  30.0000  30.0000
85   Default   382.000000   extremely   382.000000  29.0000  29.0000
20   Default  1466.000000        came  1466.000000  28.0000  28.0000
55   Default  1460.000000        went  1460.000000  27.0000  27.0000
126  Default   841.000000  experience   841.000000  26.0000  26.0000
71   Default   394.000000       food,   394.000000  25.0000  25.0000
144  Default  1240.000000         try  1240.000000  24.0000  24.0000
17   Default  2507.000000        time  2507.000000  23.0000  23.0000
43   Default  1359.000000        nice  1359.000000  22.0000  22.0000
180  Default   686.000000       fresh   686.000000  21.0000  21.0000
63   Default  3748.000000       place  3748.000000  20.0000  20.0000
9    Default   453.000000      looked   453.000000  19.0000  19.0000
328  Default   651.000000        feel   651.000000  18.0000  18.0000
177  Default   356.000000     doesn't   356.000000  17.0000  17.0000
282  Default   727.000000        work   727.000000  16.0000  16.0000
422  Default   541.000000           5   541.000000  15.0000  15.0000
379  Default   431.000000       large   431.000000  14.0000  14.0000
179  Default  3615.000000        food  3615.000000  13.0000  13.0000
593  Default   446.000000    probably   446.000000  12.0000  12.0000
378  Default   403.000000        kind   403.000000  11.0000  11.0000
445  Default   621.000000    location   621.000000  10.0000  10.0000
104  Default  1152.000000         it.  1152.000000   9.0000   9.0000
201  Default   812.000000           2   812.000000   8.0000   8.0000
88   Default  3092.000000        good  3092.000000   7.0000   7.0000
185  Default  1423.000000      little  1423.000000   6.0000   6.0000
53   Default   516.000000       vegas   516.000000   5.0000   5.0000
57   Default   476.000000    actually   476.000000   4.0000   4.0000
15   Default  1084.000000        said  1084.000000   3.0000   3.0000
340  Default   352.000000       won't   352.000000   2.0000   2.0000
19   Default   830.000000        wait   830.000000   1.0000   1.0000
..       ...          ...         ...          ...      ...      ...
104   Topic2   679.397400         it.  1152.250977   0.1661  -5.2503
179   Topic2  2002.148804        food  3615.908203   0.1032  -4.1695
15    Topic2   636.526306        said  1084.404175   0.1616  -5.3155
69    Topic2   383.462891   different   637.363770   0.1862  -5.8223
185   Topic2   822.778564      little  1423.751465   0.1460  -5.0588
88    Topic2  1709.935181        good  3092.973145   0.1016  -4.3273
68    Topic2   466.478027         bit   788.722717   0.1691  -5.6263
101   Topic2  1811.345825       great  3429.704590   0.0559  -4.2697
435   Topic2   619.371521         new  1080.724854   0.1376  -5.3428
147   Topic2   560.888855        want   977.078491   0.1393  -5.4420
225   Topic2   626.929138        know  1103.578613   0.1288  -5.3307
186   Topic2   664.460999       order  1183.165771   0.1173  -5.2725
265   Topic2   843.626648         i'm  1552.863892   0.0842  -5.0338
477   Topic2   786.549683     ordered  1445.083252   0.0861  -5.1039
35    Topic2  1057.627930         got  2047.167480   0.0339  -4.8077
205   Topic2   513.229492      better   906.662964   0.1253  -5.5308
58    Topic2   842.262390        best  1636.633301   0.0300  -5.0354
97    Topic2   903.009460       don't  1814.642822  -0.0036  -4.9658
360   Topic2  1199.650879        it's  2676.921387  -0.1083  -4.6817
330   Topic2   786.922729        love  1585.255371  -0.0060  -5.1034
354   Topic2   548.929260           &  1008.502075   0.0861  -5.4635
303   Topic2   739.511414      didn't  1512.979248  -0.0215  -5.1655
17    Topic2  1051.199097        ti

All of the yelp reviews seem to be quite similar in topic, and many of the reviews are positive.
With lambda set to one, 'food', 'great', 'good', 'place' seems to be included in most of the reviews,
no matter how many topics I split the reviews into.
The differences between topics seem to be focused on the type of food that the restaraunt serves.
With lambda set to 0.1, the groups become seperated by words like 'pizza', 'chicken' and 'burgers'.

## Stretch Goals

Complete one of more of these to push your score towards a three: 
* Incorporate named entity recognition into your analysis
* Compare vectorization methods in the classification section
* Analyze more (or all) of the yelp dataset - this one is v. hard. 
* Use a generator object on the reviews file - this would help you with the analyzing the whole dataset.
* Incorporate any of the other yelp dataset entities in your analysis (business, users, etc.)